In [1]:
import requests
import os
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import earthpy.plot as ep
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from tqdm import tqdm
from rasterio.plot import plotting_extent
import csv
import io
from contextlib import redirect_stdout
from shapely.geometry import Point


data_dir = os.path.join(os.path.expanduser('~'), 'GitHub', 'Chile-Glaciers')

/tmp/ipykernel_28796/2811974734.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
import ee
import geemap

ee.Initialize()


In [3]:
countries = gpd.read_file(os.path.join('data', 'ne_50m_admin_0_countries', 'ne_50m_admin_0_countries.shp'))

chile = countries[countries['SOVEREIGNT']=='Chile'].clip([-75,-60,-65,-15])

In [4]:
clipped_chile_path = os.path.join(data_dir, "data", "chile", "chile.shp")
chile.to_file(clipped_chile_path)

In [5]:
clipped_chile = gpd.read_file(clipped_chile_path)

In [6]:
clipped_chile.bounds

,minx,miny,maxx,maxy
0,-75.0,-55.891699,-66.435791,-17.506055


In [7]:
chile_ee = geemap.shp_to_ee(clipped_chile_path)

In [8]:
def get_NDSI_gee(geometry_path):
    """
    Calculate Normalized Difference Snow Index (NDSI) for a given geometry using Google Earth Engine.        
    """
    gee_geom = geemap.shp_to_ee(geometry_path)
    bBox = ee.Geometry.BBox(-72, -38, -71, -37)

    bBoxBounds = bBox.bounds()

    # Initialize an ImageCollection object with the desired Landsat dataset
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

    # Filter the Landsat ImageCollection by the geometry, date range and sort by cloud cover
    # Select the first image (the one with the least cloud cover)
    image = ee.Image(
        l8.filterBounds(gee_geom)
        .filterDate('2022-01-01', '2022-03-30')
        .filterMetadata('CLOUD_COVER', 'less_than', 10)
        .mean()
    )
    
    # Calculate NDSI from the selected image, rename the resulting image to 'NDSI'
    ndsi = image.normalizedDifference(['B3', 'B6']).rename('NDSI')
    red_band = image.select('B4')
    ndsi = ndsi.updateMask(red_band.gte(.05));
    ndsi_clip = ndsi.clip(gee_geom).unmask()
    return ndsi_clip.gte(.4)
    boxcar = ee.Kernel.square(21, 'pixels', False)

    # Clip the NDSI image to the geometry and unmask the result
    return ndsi_clip.convolve(boxcar)


def export_open_gee(gee_ndsi, geometry, out_dir):
    """
    Export an Earth Engine image to a GeoTIFF file and open it with RioXarray.

    """
    # set a trap and redirect stdout
    trap = io.StringIO()

    # Create a file name
    filename = os.path.join(
        out_dir, 'chile_ndsi.tif')

    # Get the bounding box coordinates of the catchment feature
    bounds = list(geometry.bounds.values[0])

    # If the exported file already exists, remove it
    if os.path.exists(filename):
        os.remove(filename)

    # Export the Earth Engine image to the GeoTIFF file
#     with redirect_stdout(trap):
    geemap.ee_export_image(
        gee_ndsi, filename=filename, scale=30, region=bounds, file_per_band=False, crs="EPSG:4326"
    )

    # Open the exported GeoTIFF file into an xarray dataarray
    return rxr.open_rasterio(filename, mask=True).rio.clip(geometry)

In [9]:
ndsi_gee = get_NDSI_gee(clipped_chile_path)

In [10]:
Map = geemap.Map()
Map.addLayer(ndsi_gee)
Map.setCenter(-71, -37, 8)
# Map.addLayer(chile_ee)

Map

Map(center=[-37, -71], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…